In [2]:
# === Cell 1: Core model + preprocessing (ORB-friendly) + verify (affine RANSAC) + inline display ===
import cv2, os, numpy as np
import matplotlib.pyplot as plt

# ---------- Haar cascade ----------
def load_face_cascade():
    candidates = [
        "models/haarcascade_frontalface_default.xml",
        os.path.join(os.getcwd(), "haarcascade_frontalface_default.xml"),
        os.path.join(cv2.data.haarcascades, "haarcascade_frontalface_default.xml"),
    ]
    for p in candidates:
        if os.path.exists(p):
            c = cv2.CascadeClassifier(p)
            if not c.empty():
                return c
    raise IOError("Haar cascade not found. Put 'haarcascade_frontalface_default.xml' in ./models/ or project root.")

_FACE_CASCADE = load_face_cascade()

def detect_largest_face_bgr(bgr_img, scaleFactor=1.1, minNeighbors=5):
    if bgr_img is None: return None, None
    gray = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
    faces = _FACE_CASCADE.detectMultiScale(gray, scaleFactor, minNeighbors)
    if len(faces) == 0: return None, None
    (x, y, w, h) = max(faces, key=lambda r: r[2]*r[3])
    return (x, y, w, h), bgr_img[y:y+h, x:x+w]

# ---------- Preprocessing (ORB-friendly: no blur; preserve corners/edges) ----------
def preprocess_face(bgr_face, out_size=(160,160)):
    gray = cv2.cvtColor(bgr_face, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    eq = clahe.apply(gray)
    # IMPORTANT: do NOT blur for ORB (keeps corner strength)
    resized = cv2.resize(eq, out_size, interpolation=cv2.INTER_AREA)
    return resized  # grayscale

# ---------- ORB features (stronger config) ----------
def extract_orb_features(gray_img, nfeatures=2000):
    # More features, more robust descriptors; lower FAST threshold to pick up low-contrast corners
    orb = cv2.ORB_create(
        nfeatures=nfeatures,
        scaleFactor=1.2,
        nlevels=8,
        edgeThreshold=15,
        firstLevel=0,
        WTA_K=4,
        scoreType=cv2.ORB_HARRIS_SCORE,
        patchSize=31,
        fastThreshold=7
    )
    kps, des = orb.detectAndCompute(gray_img, None)
    return kps, des

# ---------- Matching & scoring (ratio test + affine RANSAC inliers) ----------
def match_and_score(des1, des2, ratio=0.78, require_model=True, model="affine"):
    """
    Returns (score, good_matches, inlier_mask)
      - score = #inliers if a geometric model is estimated; else len(good_matches)
      - model: "affine" (preferred for faces) or "homography"
    """
    if des1 is None or des2 is None or len(des1) < 2 or len(des2) < 2:
        return 0, [], None

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    knn = bf.knnMatch(des1, des2, k=2)
    good = [m for m, n in knn if m.distance < ratio * n.distance]

    inlier_mask = None
    if require_model and len(good) >= 6:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        if model == "affine":
            # Similarity/affine model is more realistic for faces than a full homography
            M, mask = cv2.estimateAffinePartial2D(
                src_pts, dst_pts, method=cv2.RANSAC, ransacReprojThreshold=4.0, maxIters=2000, confidence=0.99
            )
        else:
            H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if mask is not None:
            inlier_mask = mask.ravel().tolist()
            return int(np.sum(mask)), good, inlier_mask

    return len(good), good, inlier_mask

# ---------- Verify wrapper (affine inliers preferred; fallback to good-match count) ----------
def verify_faces(ref_gray, cap_gray, threshold_inliers=10, threshold_goods=18, force_model="affine"):
    """
    Returns: decision (bool), score (int), dbg (dict)
      - decision uses inlier count if a model was found; otherwise good-match count.
      - thresholds are set to be reasonable for your images; tune if needed.
    """
    global kp1, kp2
    kp1, des1 = extract_orb_features(ref_gray)
    kp2, des2 = extract_orb_features(cap_gray)

    score, good_matches, inlier_mask = match_and_score(
        des1, des2, ratio=0.78, require_model=True, model=force_model
    )

    used_metric = "inliers" if inlier_mask is not None else "good_matches"
    threshold = threshold_inliers if used_metric == "inliers" else threshold_goods
    decision = score >= threshold

    dbg = {
        "used_metric": used_metric,
        "threshold": threshold,
        "score": score,
        "num_kp_ref": 0 if kp1 is None else len(kp1),
        "num_kp_cap": 0 if kp2 is None else len(kp2),
        "good_matches": len(good_matches),
        "inlier_mask": inlier_mask,
        "good_matches_list": good_matches
    }
    return decision, score, dbg

# ---------- Draw matches (keeps your inline visualization flow) ----------
def draw_matches(ref_gray, cap_gray, matches, inlier_mask=None):
    flags = cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    if inlier_mask is not None:
        inlier_matches = [m for m, keep in zip(matches, inlier_mask) if keep]
        return cv2.drawMatches(ref_gray, kp1, cap_gray, kp2, inlier_matches, None, flags=flags)
    return cv2.drawMatches(ref_gray, kp1, cap_gray, kp2, matches, None, flags=flags)

# ---------- Inline display helpers (Notebook output) ----------
def _to_rgb(img):
    if img.ndim == 2:      # gray
        return img
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def show_pair(title, left_img, right_img, left_label="Left", right_label="Right", figsize=(9,4)):
    plt.figure(figsize=figsize)
    plt.suptitle(title, fontsize=16, fontweight="bold")
    plt.subplot(1,2,1); plt.imshow(_to_rgb(left_img), cmap=None if left_img.ndim==3 else "gray")
    plt.title(left_label); plt.axis('off')
    plt.subplot(1,2,2); plt.imshow(_to_rgb(right_img), cmap=None if right_img.ndim==3 else "gray")
    plt.title(right_label); plt.axis('off')
    plt.show()

def show_single(title, img, figsize=(6,6)):
    plt.figure(figsize=figsize)
    plt.title(title, fontsize=14, fontweight="bold")
    plt.imshow(_to_rgb(img), cmap=None if img.ndim==3 else "gray")
    plt.axis('off')
    plt.show()


In [3]:
# === Cell 2: Live camera capture (returns largest face crop in BGR) ===
import cv2

def capture_and_verify_loop(ref_image=None):
    cap = cv2.VideoCapture(0)
    captured_face = None

    print("Live camera: SPACE=capture, Q/ESC=quit")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: camera read failed.")
            break

        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = _FACE_CASCADE.detectMultiScale(gray, 1.1, 4)

        if len(faces) > 0:
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
                cv2.putText(frame, "Face Detected", (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow("Live Camera Feed", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord(' '):  # capture
            if len(faces) == 0:
                print("No face detected. Try again.")
                continue
            x, y, w, h = max(faces, key=lambda r: r[2]*r[3])
            captured_face = frame[y:y+h, x:x+w]
            break
        elif key in (ord('q'), 27):  # 'q' or ESC
            print("Exit.")
            break

    cap.release()
    cv2.destroyAllWindows()
    return captured_face


In [ ]:
# === Cell 3: With camera — inline compares + printed summary ===
import cv2

# 1) Load reference original (change to your file)
ref_path = r"data/faces/Phua Hong Yip/jia ling.jpg"
ref_bgr = cv2.imread(ref_path)
if ref_bgr is None:
    raise FileNotFoundError(f"Cannot read reference image: {ref_path}")

# 2) Detect reference face
_, ref_face_bgr = detect_largest_face_bgr(ref_bgr)
if ref_face_bgr is None:
    raise RuntimeError("No face detected in reference image.")

# 3) Capture live face
cap_face_bgr = capture_and_verify_loop()
if cap_face_bgr is None:
    raise RuntimeError("Capture canceled or no face captured.")

# 4) Preprocess both
ref_face_pp = preprocess_face(ref_face_bgr)
cap_face_pp = preprocess_face(cap_face_bgr)

# 5) Show inline compares (no windows)
show_pair("Compare 1: Reference — Original vs Preprocessed",
          ref_face_bgr, ref_face_pp,
          "Reference (Original)", "Reference (Preprocessed)")

show_pair("Compare 2: Capture — Original vs Preprocessed",
          cap_face_bgr, cap_face_pp,
          "Capture (Original)", "Capture (Preprocessed)")

show_pair("Compare 3: Preprocessed — Reference vs Capture",
          ref_face_pp, cap_face_pp,
          "Ref (Preprocessed)", "Cap (Preprocessed)")

# 6) Verify and show matches (preprocessed vs preprocessed)
decision, score, dbg = verify_faces(ref_face_pp, cap_face_pp)
match_vis = draw_matches(ref_face_pp, cap_face_pp, dbg['good_matches_list'], dbg['inlier_mask'])
show_single("Compare 4: Verification — ORB Matches / Inliers (Preprocessed Ref vs Cap)", match_vis)

# 7) Print concise summary
print("=========== FACE VERIFY SUMMARY ===========")
print(f"Metric used      : {dbg['used_metric']}")
print(f"Threshold        : {dbg['threshold']}")
print(f"Score            : {dbg['score']}")
print(f"Keypoints (ref)  : {dbg['num_kp_ref']}")
print(f"Keypoints (cap)  : {dbg['num_kp_cap']}")
print(f"Good matches     : {dbg['good_matches']}")
print(f"Decision         : {'MATCH ✅' if decision else 'NOT MATCH ❌'}")
print("===========================================")


In [4]:
# === Cell 4: ORB Verification/Identification Evaluation on Your Dataset ===
import os, json, random, itertools, numpy as np, cv2
from collections import defaultdict
from pathlib import Path

try:
    from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
    HAVE_SK = True
except Exception:
    HAVE_SK = False

# ---------- Config ----------
DATA_ROOT      = r"C:\Users\jians\Documents\GitHub\SMART-Barcode-Scanner-and-Face-Recognition\data\data_proc"     # <- change if needed
VAL_DIRNAME    = "val"             # expects DATA_ROOT/val/<ID>/*.jpg
TEST_DIRNAME   = "test"            # expects DATA_ROOT/test/<ID>/*.jpg
TRAIN_DIRNAME  = "train"           # for identification gallery
ALLOWED_EXT    = {".jpg", ".jpeg", ".png", ".bmp"}

# Pair sampling caps (set to None for 'all')
POS_PER_ID_VAL   = 20
NEG_PER_ID_VAL   = 25
POS_PER_ID_TEST  = None
NEG_PER_ID_TEST  = 25

RANSAC_MODEL     = "affine"  # "affine" (recommended) or "homography"

OUT_DIR          = "orb_eval_out"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# ---------- Helpers ----------
def list_id_images(split_dir):
    """
    Returns dict: {id_name: [image_paths...]}
    """
    out = {}
    p = Path(split_dir)
    if not p.exists():
        raise FileNotFoundError(f"Split not found: {split_dir}")
    for sub in sorted(p.iterdir()):
        if sub.is_dir():
            imgs = [str(x) for x in sorted(sub.rglob("*")) if x.suffix.lower() in ALLOWED_EXT]
            if imgs:
                out[sub.name] = imgs
    return out

def load_face_gray_160(img_path):
    """
    Loads BGR, detects largest face, preprocesses to your ORB-friendly grayscale 160x160.
    Returns gray image or None if detection fails.
    """
    bgr = cv2.imread(img_path)
    if bgr is None:
        return None
    _, face_bgr = detect_largest_face_bgr(bgr)
    if face_bgr is None:
        return None
    return preprocess_face(face_bgr, out_size=(160,160))  # uses your function

def orb_pair_score(gray1, gray2, model=RANSAC_MODEL):
    """
    Returns (score:int, used_metric:str) where score = #inliers if model found,
    otherwise #good_matches. Uses your ORB + match pipeline.
    """
    global kp1, kp2  # required by your match_and_score() for drawing/mapping
    kp1, des1 = extract_orb_features(gray1)
    kp2, des2 = extract_orb_features(gray2)
    score, good_matches, inlier_mask = match_and_score(
        des1, des2, ratio=0.78, require_model=True, model=model
    )
    used = "inliers" if inlier_mask is not None else "good_matches"
    return int(score), used

def make_pairs(id2imgs, pos_cap=None, neg_cap=None):
    """
    Build (imgA, imgB, label) pairs.
     - Positive: within each ID.
     - Negative: across different IDs.
    Caps are per-ID (so it scales on big datasets).
    """
    rng = random.Random(1337)
    pairs = []

    # Positives
    for _id, imgs in id2imgs.items():
        if len(imgs) < 2: 
            continue
        combos = list(itertools.combinations(imgs, 2))
        rng.shuffle(combos)
        if pos_cap is not None:
            combos = combos[:pos_cap]
        pairs.extend([(a, b, 1) for (a, b) in combos])

    # Negatives
    ids = list(id2imgs.keys())
    for i, id_a in enumerate(ids):
        imgs_a = id2imgs[id_a]
        others = ids[:i] + ids[i+1:]
        if not others or not imgs_a:
            continue
        # choose some 'b' identities randomly
        rng.shuffle(others)
        neg_samples = []
        for id_b in others:
            imgs_b = id2imgs[id_b]
            if not imgs_b: 
                continue
            # one random cross-pair per other-id to diversify
            a = rng.choice(imgs_a)
            b = rng.choice(imgs_b)
            neg_samples.append((a, b, 0))
        rng.shuffle(neg_samples)
        if neg_cap is not None:
            neg_samples = neg_samples[:neg_cap]
        pairs.extend(neg_samples)

    rng.shuffle(pairs)
    return pairs

def score_pairs(pairs):
    """
    Returns (scores:list, labels:list, bad:list_of_indices)
    Skips pairs where face detection fails; records their indices in 'bad'.
    """
    scores, labels, bad = [], [], []
    for idx, (pa, pb, y) in enumerate(pairs):
        ga = load_face_gray_160(pa)
        gb = load_face_gray_160(pb)
        if ga is None or gb is None:
            bad.append(idx)
            continue
        s, _ = orb_pair_score(ga, gb)
        scores.append(s); labels.append(y)
    return np.array(scores, dtype=float), np.array(labels, dtype=int), bad

def find_tau_star(scores, labels):
    """
    Find threshold τ* that minimizes |FNR - FPR| (approx. EER point).
    Returns tau_star, EER, stats dict.
    """
    if len(scores) == 0:
        return 0.0, 1.0, {"note": "empty scores"}
    uniq = np.unique(scores)
    # Consider midpoints between sorted unique scores to be safer
    cuts = np.concatenate(([uniq[0]-1], (uniq[:-1]+uniq[1:])/2.0, [uniq[-1]+1]))
    best_tau, best_gap, best_eer = None, 1.0, 1.0
    for tau in cuts:
        preds = (scores >= tau).astype(int)
        TP = ((preds==1) & (labels==1)).sum()
        TN = ((preds==0) & (labels==0)).sum()
        FP = ((preds==1) & (labels==0)).sum()
        FN = ((preds==0) & (labels==1)).sum()
        P  = max(1, (labels==1).sum())
        N  = max(1, (labels==0).sum())
        TPR = TP / P
        FPR = FP / N
        FNR = 1 - TPR
        gap = abs(FNR - FPR)
        eer = (FNR + FPR)/2.0
        if gap < best_gap:
            best_gap, best_tau, best_eer = gap, float(tau), float(eer)
    stats = {"gap": best_gap}
    return best_tau, best_eer, stats

def bin_metrics(scores, labels, tau):
    preds = (scores >= tau).astype(int)
    TP = ((preds==1) & (labels==1)).sum()
    TN = ((preds==0) & (labels==0)).sum()
    FP = ((preds==1) & (labels==0)).sum()
    FN = ((preds==0) & (labels==1)).sum()

    acc = (TP + TN) / max(1, len(labels))
    prec = TP / max(1, (TP + FP))
    rec  = TP / max(1, (TP + FN))
    f1   = 0.0 if (prec+rec)==0 else 2*prec*rec/(prec+rec)
    out = {
        "ACC": round(float(acc), 6),
        "Precision": round(float(prec), 6),
        "Recall": round(float(rec), 6),
        "F1": round(float(f1), 6),
        "TP": int(TP), "TN": int(TN), "FP": int(FP), "FN": int(FN),
        "Support_Pos": int((labels==1).sum()),
        "Support_Neg": int((labels==0).sum())
    }
    if HAVE_SK:
        try:
            out["ROC_AUC"] = round(float(roc_auc_score(labels, scores)), 6)
        except Exception:
            out["ROC_AUC"] = None
    else:
        out["ROC_AUC"] = None
    return out, preds

def save_json(obj, path):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

# ---------- Identification (optional) ----------
def build_gallery(id2imgs_train, per_id=1):
    """
    Returns dict {id_name: [preprocessed_gray_images]} using up to per_id images per identity.
    """
    gal = {}
    for _id, paths in id2imgs_train.items():
        picked = paths[:per_id]
        grays = []
        for p in picked:
            g = load_face_gray_160(p)
            if g is not None:
                grays.append(g)
        if grays:
            gal[_id] = grays
    return gal

def identify_top1(gallery, id2imgs_test):
    """
    For each test image, compare to gallery of each ID, take max score; choose ID with max.
    Returns top1 accuracy and per-ID stats.
    """
    total, correct = 0, 0
    per_id = defaultdict(lambda: {"n":0, "correct":0})

    ids = list(id2imgs_test.keys())
    for true_id in ids:
        for p in id2imgs_test[true_id]:
            g = load_face_gray_160(p)
            if g is None:
                continue
            best_id, best_score = None, -1
            for gid, gimgs in gallery.items():
                # score against all gallery imgs for that ID, take max
                scores = []
                for gg in gimgs:
                    s, _ = orb_pair_score(g, gg)
                    scores.append(s)
                if scores:
                    smax = max(scores)
                    if smax > best_score:
                        best_score, best_id = smax, gid
            if best_id is None:
                continue
            total += 1
            if best_id == true_id:
                correct += 1
                per_id[true_id]["correct"] += 1
            per_id[true_id]["n"] += 1

    acc = 0.0 if total==0 else correct/total
    return acc, total, correct, per_id

# ---------- Run full evaluation ----------
def evaluate_and_report(
    data_root=DATA_ROOT,
    val_dirname=VAL_DIRNAME,
    test_dirname=TEST_DIRNAME,
    train_dirname=TRAIN_DIRNAME,
    pos_per_id_val=POS_PER_ID_VAL,
    neg_per_id_val=NEG_PER_ID_VAL,
    pos_per_id_test=POS_PER_ID_TEST,
    neg_per_id_test=NEG_PER_ID_TEST,
    gallery_per_id=1,
    do_identification=True
):
    # 1) Index splits
    val_ids  = list_id_images(os.path.join(data_root, val_dirname))
    test_ids = list_id_images(os.path.join(data_root, test_dirname))
    print(f"[VAL]  identities: {len(val_ids)}")
    print(f"[TEST] identities: {len(test_ids)}")

    # 2) Build validation pairs → choose tau*
    pairs_val = make_pairs(val_ids, pos_cap=pos_per_id_val, neg_cap=neg_per_id_val)
    print(f"[VAL] pairs: {len(pairs_val)} (pos_cap={pos_per_id_val}, neg_cap={neg_per_id_val})")
    scores_val, labels_val, bad_val = score_pairs(pairs_val)
    print(f"[VAL] usable: {len(scores_val)}, skipped (no face): {len(bad_val)}")
    tau_star, eer, tau_meta = find_tau_star(scores_val, labels_val)
    val_metrics, _ = bin_metrics(scores_val, labels_val, tau_star)
    print(f"[VAL] τ*={tau_star:.3f} | EER≈{eer:.4f} | ACC={val_metrics['ACC']:.4f} | F1={val_metrics['F1']:.4f} | ROC_AUC={val_metrics['ROC_AUC']}")

    # 3) Test pairs → fixed tau*
    pairs_test = make_pairs(test_ids, pos_cap=pos_per_id_test, neg_cap=neg_per_id_test)
    print(f"[TEST] pairs: {len(pairs_test)}")
    scores_test, labels_test, bad_test = score_pairs(pairs_test)
    print(f"[TEST] usable: {len(scores_test)}, skipped (no face): {len(bad_test)}")
    test_metrics, preds_test = bin_metrics(scores_test, labels_test, tau_star)
    print(f"[TEST] ACC={test_metrics['ACC']:.4f} | Precision={test_metrics['Precision']:.4f} | Recall={test_metrics['Recall']:.4f} | F1={test_metrics['F1']:.4f} | ROC_AUC={test_metrics['ROC_AUC']}")
    if HAVE_SK:
        print("\n[TEST] Classification report:")
        try:
            print(classification_report(labels_test, preds_test, target_names=["Different","Same"]))
        except Exception:
            pass
        try:
            cm = confusion_matrix(labels_test, preds_test)
            print("[TEST] Confusion Matrix:\n", cm)
        except Exception:
            pass

    # 4) Save artifacts
    save_json({
        "tau_star": tau_star,
        "eer": eer,
        "val_metrics": val_metrics,
        "test_metrics": test_metrics,
        "notes": {"tau_search": tau_meta}
    }, os.path.join(OUT_DIR, "orb_threshold_and_metrics.json"))

    np.save(os.path.join(OUT_DIR, "scores_val.npy"),  scores_val)
    np.save(os.path.join(OUT_DIR, "labels_val.npy"),  labels_val)
    np.save(os.path.join(OUT_DIR, "scores_test.npy"), scores_test)
    np.save(os.path.join(OUT_DIR, "labels_test.npy"), labels_test)

    # 5) Optional identification (Top-1)
    id_report = None
    if do_identification:
        train_ids = list_id_images(os.path.join(data_root, train_dirname))
        gallery = build_gallery(train_ids, per_id=gallery_per_id)
        id_acc, total, correct, per_id = identify_top1(gallery, test_ids)
        id_report = {
            "top1_acc": round(float(id_acc), 6),
            "total_probes": int(total),
            "correct": int(correct),
            "per_id": {k: {"n": v["n"], "correct": v["correct"]} for k, v in per_id.items()}
        }
        save_json(id_report, os.path.join(OUT_DIR, "identification_report.json"))
        print(f"\n[IDENTIFY] Gallery/train per-ID={gallery_per_id} → Top-1 ACC={id_acc:.4f} ({correct}/{total})")

    return tau_star, eer, val_metrics, test_metrics, id_report

# ---- Run (edit paths/caps if needed) ----
tau_star, eer, val_metrics, test_metrics, id_report = evaluate_and_report(
    data_root=DATA_ROOT,
    val_dirname=VAL_DIRNAME,
    test_dirname=TEST_DIRNAME,
    train_dirname=TRAIN_DIRNAME,
    pos_per_id_val=POS_PER_ID_VAL,
    neg_per_id_val=NEG_PER_ID_VAL,
    pos_per_id_test=POS_PER_ID_TEST,
    neg_per_id_test=NEG_PER_ID_TEST,
    gallery_per_id=1,          # 1 ref image per ID in train for identification
    do_identification=True
)
print("\nSaved:")
print(f" - {OUT_DIR}/orb_threshold_and_metrics.json")
print(f" - {OUT_DIR}/scores_val.npy, labels_val.npy")
print(f" - {OUT_DIR}/scores_test.npy, labels_test.npy")
if id_report is not None:
    print(f" - {OUT_DIR}/identification_report.json")


[VAL]  identities: 31
[TEST] identities: 31
[VAL] pairs: 1362 (pos_cap=20, neg_cap=25)
[VAL] usable: 1152, skipped (no face): 210
[VAL] τ*=6.500 | EER≈0.3818 | ACC=0.6120 | F1=0.5947 | ROC_AUC=0.671571
[TEST] pairs: 3360
[TEST] usable: 2750, skipped (no face): 610
[TEST] ACC=0.6564 | Precision=0.8530 | Recall=0.6774 | F1=0.7551 | ROC_AUC=0.692222

[TEST] Classification report:
              precision    recall  f1-score   support

   Different       0.33      0.58      0.42       599
        Same       0.85      0.68      0.76      2151

    accuracy                           0.66      2750
   macro avg       0.59      0.63      0.59      2750
weighted avg       0.74      0.66      0.68      2750

[TEST] Confusion Matrix:
 [[ 348  251]
 [ 694 1457]]

[IDENTIFY] Gallery/train per-ID=1 → Top-1 ACC=0.2306 (83/360)

Saved:
 - orb_eval_out/orb_threshold_and_metrics.json
 - orb_eval_out/scores_val.npy, labels_val.npy
 - orb_eval_out/scores_test.npy, labels_test.npy
 - orb_eval_out/identifica